In [2]:
import numpy as np  
import pandas as pd 
import re           
from keras.preprocessing.text import Tokenizer 
from keras.preprocessing.sequence import pad_sequences
from nltk.corpus import stopwords
from tensorflow.keras.layers import Input,  LSTM, Embedding, Dense, Concatenate, TimeDistributed, Bidirectional
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.models import Model
from Attention import AttentionLayer

In [3]:
dataset = pd.read_csv("./news_summary.csv",nrows=70000)

In [4]:

contraction_mapping = {"ain't": "is not", "aren't": "are not","can't": "cannot", "'cause": "because", "could've": "could have", "couldn't": "could not",

                           "didn't": "did not", "doesn't": "does not", "don't": "do not", "hadn't": "had not", "hasn't": "has not", "haven't": "have not",

                           "he'd": "he would","he'll": "he will", "he's": "he is", "how'd": "how did", "how'd'y": "how do you", "how'll": "how will", "how's": "how is",

                           "I'd": "I would", "I'd've": "I would have", "I'll": "I will", "I'll've": "I will have","I'm": "I am", "I've": "I have", "i'd": "i would",

                           "i'd've": "i would have", "i'll": "i will",  "i'll've": "i will have","i'm": "i am", "i've": "i have", "isn't": "is not", "it'd": "it would",

                           "it'd've": "it would have", "it'll": "it will", "it'll've": "it will have","it's": "it is", "let's": "let us", "ma'am": "madam",

                           "mayn't": "may not", "might've": "might have","mightn't": "might not","mightn't've": "might not have", "must've": "must have",

                           "mustn't": "must not", "mustn't've": "must not have", "needn't": "need not", "needn't've": "need not have","o'clock": "of the clock",

                           "oughtn't": "ought not", "oughtn't've": "ought not have", "shan't": "shall not", "sha'n't": "shall not", "shan't've": "shall not have",

                           "she'd": "she would", "she'd've": "she would have", "she'll": "she will", "she'll've": "she will have", "she's": "she is",

                           "should've": "should have", "shouldn't": "should not", "shouldn't've": "should not have", "so've": "so have","so's": "so as",

                           "this's": "this is","that'd": "that would", "that'd've": "that would have", "that's": "that is", "there'd": "there would",

                           "there'd've": "there would have", "there's": "there is", "here's": "here is","they'd": "they would", "they'd've": "they would have",

                           "they'll": "they will", "they'll've": "they will have", "they're": "they are", "they've": "they have", "to've": "to have",

                           "wasn't": "was not", "we'd": "we would", "we'd've": "we would have", "we'll": "we will", "we'll've": "we will have", "we're": "we are",

                           "we've": "we have", "weren't": "were not", "what'll": "what will", "what'll've": "what will have", "what're": "what are",

                           "what's": "what is", "what've": "what have", "when's": "when is", "when've": "when have", "where'd": "where did", "where's": "where is",

                           "where've": "where have", "who'll": "who will", "who'll've": "who will have", "who's": "who is", "who've": "who have",

                           "why's": "why is", "why've": "why have", "will've": "will have", "won't": "will not", "won't've": "will not have",

                           "would've": "would have", "wouldn't": "would not", "wouldn't've": "would not have", "y'all": "you all",

                           "y'all'd": "you all would","y'all'd've": "you all would have","y'all're": "you all are","y'all've": "you all have",

                           "you'd": "you would", "you'd've": "you would have", "you'll": "you will", "you'll've": "you will have",

                           "you're": "you are", "you've": "you have"}


In [5]:
stop_words = set(stopwords.words('english')) 
def text_cleaner(text):
    newString = text.lower()
    newString = re.sub(r'\([^)]*\)', '', newString)
    newString = re.sub('"','', newString)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString) 
    tokens = [w for w in newString.split() if not w in stop_words]
    long_words=[]
    for i in tokens:
        if len(i)>=3:                  #removing short word
            long_words.append(i)
    return(" ".join(long_words)).strip()

cleaned_text = []
for t in dataset['text']:
    cleaned_text.append(text_cleaner(t))

In [ ]:
def headline_cleaner(text):
    newString = re.sub('"','', text)
    newString = ' '.join([contraction_mapping[t] if t in contraction_mapping else t for t in newString.split(" ")])    
    newString = re.sub(r"'s\b","",newString)
    newString = re.sub("[^a-zA-Z]", " ", newString)
    newString = newString.lower() 
    tokens = newString.split()
    newString = ''
    for i in tokens:
        if len(i)>1:                                 
            newString=newString+i+' '  
    return newString 

#Call the above function
cleaned_headline = []
for t in dataset['headlines']:
    cleaned_headline.append(headline_cleaner(t))


In [ ]:
dataset['cleaned_text']=cleaned_text
dataset['cleaned_headline']=cleaned_headline
dataset['cleaned_headline'].replace('', np.nan, inplace=True)
dataset.dropna(axis=0,inplace=True)

dataset['cleaned_headline'] = dataset['cleaned_headline'].apply(lambda x : '_START_ '+ x + ' _END_')

word_count_text = []
word_count_headline = []

for i in dataset['cleaned_text']:
    word_count_text.append(len(i.split(' ')))

for i in dataset['cleaned_headline']:
    word_count_headline.append(len(i.split(' ')))


max_len_text = max(word_count_text)
max_len_headline = max(word_count_headline)

In [ ]:
from sklearn.model_selection import train_test_split
x_tr,x_val,y_tr,y_val=train_test_split(dataset['cleaned_text'],dataset['cleaned_headline'],test_size=0.1,random_state=0,shuffle=True) 


# tokenization
# for text
x_token = Tokenizer()
# fit_on_texts method build word_index =>  creates the vocabulary index based on word frequency
x_token.fit_on_texts(list(x_tr))

x_tr = x_token.texts_to_sequences(x_tr)
x_val = x_token.texts_to_sequences(x_val)

x_tr = pad_sequences(x_tr,maxlen=max_len_text,padding='post')
x_val = pad_sequences(x_val,maxlen=max_len_text,padding='post')

x_voc_size = len(x_token.word_index) + 1

# for headlines
y_token = Tokenizer()
y_token.fit_on_texts(list(y_tr))

y_tr = y_token.texts_to_sequences(y_tr)
y_val = y_token.texts_to_sequences(y_val)

y_tr = pad_sequences(y_tr,maxlen=max_len_headline,padding='post')
y_val = pad_sequences(y_val,maxlen=max_len_headline,padding='post')

y_voc_size = len(y_token.word_index) + 1

x_tr.shape,y_tr.shape

In [ ]:
from keras import backend as k
k.clear_session()

latent_dim = 300
embedding_dim = 100

#encoder
enc_inp = Input(shape=(max_len_text,))

#embedding layer
enc_emb = Embedding(x_voc_size,embedding_dim,trainable=True)(enc_inp)
# bidirectional lstm
encoder_1 = Bidirectional(LSTM(latent_dim,return_sequences=True,return_state=True))
enc_out_1, frw_h1, frw_c1, bck_h1,bck_c1 = encoder_1(enc_emb)

encoder_2 = Bidirectional(LSTM(latent_dim,return_sequences=True,return_state=True))
enc_out_2, frw_h2, frw_c2, bck_h2,bck_c2 = encoder_2(enc_out_1)

encoder = Bidirectional(LSTM(latent_dim,return_state=True, return_sequences=True))
enc_out, frw_h, frw_c, bck_h, bck_c  = encoder(enc_out_2)
state_h = Concatenate()([frw_h,bck_h])
state_c = Concatenate()([frw_c,bck_c])
encoder_states = [state_h,state_c]